<a href="https://colab.research.google.com/github/NapoliD/Anomalias/blob/main/Analytics_v1_Metricas_Duras_Sensores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Si lo ejecutan en drive aplican este codigo para cargar su drive
from google.colab import drive
drive.mount('/content/drive/') 

Mounted at /content/drive/


In [2]:
# actualizar entorno en donde se encuentra el CSV, si es local o su es en drive dejo el ejemplo
import os
os.chdir('/content/drive/My Drive/')

In [3]:
# Cargamos todas las bibliotecas
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)
import statistics
from datetime import datetime as date

In [4]:
# Modelo no supervisado
from sklearn.ensemble import IsolationForest


Cargamos Data

In [26]:
# Cambiar nombre del archivo
data = pd.read_csv('3 SENSORES.csv',sep = ';',encoding='latin-1',decimal=',',thousands='.')

data.head(10)


,time,_p_sensor,value
0,2020-11-30T15:49:04.229Z,AsAxgmNPfd,86
1,2020-11-30T15:49:19.661Z,yyotRBlw9E,13
2,2020-11-30T15:49:19.727Z,mINGiatYa7,127
3,2020-11-30T15:50:20.220Z,AsAxgmNPfd,86
4,2020-11-30T15:51:05.403Z,mINGiatYa7,65
5,2020-11-30T15:51:05.405Z,yyotRBlw9E,12
6,2020-11-30T15:53:10.207Z,AsAxgmNPfd,87
7,2020-11-30T15:53:10.376Z,mINGiatYa7,188
8,2020-11-30T15:53:10.395Z,yyotRBlw9E,12
9,2020-11-30T15:54:25.289Z,AsAxgmNPfd,86


In [ ]:
# Cantidad de datos
data.dtypes

time         object
_p_sensor    object
value         int64
dtype: object

In [ ]:
data.shape

(7607, 19)

In [14]:
#Creamos columna fecha
uno=data.time.str.split('T',expand=True)
uno=uno.rename(columns={0: 'dia' , 1:'hora'})
uno.head(5)

,dia,hora
0,2020-11-30,15:49:04.229Z
1,2020-11-30,15:49:19.661Z
2,2020-11-30,15:49:19.727Z
3,2020-11-30,15:50:20.220Z
4,2020-11-30,15:51:05.403Z


Tranformo hora

In [27]:
uno=data.time.str.split('T',expand=True)
uno=uno.rename(columns={0: 'dia' , 1:'hora'})
data['Fecha'] = pd.to_datetime(uno['dia'])#, format='%Y/%m/%d') 
data['day'] = data['Fecha'].dt.day


In [28]:
uno=data.time.str.split('T',expand=True)
uno=uno.rename(columns={0: 'dia' , 1:'hora'})
dos=uno.dia.str.split('-',expand=True)
dos=dos.rename(columns={0: 'anio' , 1:'mes', 2:'dia'})
tres=uno.hora.str.split(':',expand=True)
tres=tres.rename(columns={0: 'hora' , 1:'minutos', 2:'segundos'})
tres.head()

,hora,minutos,segundos
0,15,49,04.229Z
1,15,49,19.661Z
2,15,49,19.727Z
3,15,50,20.220Z
4,15,51,05.403Z


In [29]:
# Unimos todos los datos
data=pd.concat([data.reset_index(drop=True),
                      dos.reset_index(drop=True),tres.reset_index(drop=True) ], axis=1)

data['dia']=data['dia'].astype('int')
data['hora']=data['hora'].astype('int')

data['mes']=data['mes'].astype('int')
data.head(10)


,time,_p_sensor,value,Fecha,day,anio,mes,dia,hora,minutos,segundos
0,2020-11-30T15:49:04.229Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,49,04.229Z
1,2020-11-30T15:49:19.661Z,yyotRBlw9E,13,2020-11-30,30,2020,11,30,15,49,19.661Z
2,2020-11-30T15:49:19.727Z,mINGiatYa7,127,2020-11-30,30,2020,11,30,15,49,19.727Z
3,2020-11-30T15:50:20.220Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,50,20.220Z
4,2020-11-30T15:51:05.403Z,mINGiatYa7,65,2020-11-30,30,2020,11,30,15,51,05.403Z
5,2020-11-30T15:51:05.405Z,yyotRBlw9E,12,2020-11-30,30,2020,11,30,15,51,05.405Z
6,2020-11-30T15:53:10.207Z,AsAxgmNPfd,87,2020-11-30,30,2020,11,30,15,53,10.207Z
7,2020-11-30T15:53:10.376Z,mINGiatYa7,188,2020-11-30,30,2020,11,30,15,53,10.376Z
8,2020-11-30T15:53:10.395Z,yyotRBlw9E,12,2020-11-30,30,2020,11,30,15,53,10.395Z
9,2020-11-30T15:54:25.289Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,54,25.289Z


In [18]:
data.head(10)


,time,_p_sensor,value,Fecha,day,anio,mes,dia,hora,minutos,segundos
0,2020-11-30T15:49:04.229Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,49,04.229Z
1,2020-11-30T15:49:19.661Z,yyotRBlw9E,13,2020-11-30,30,2020,11,30,15,49,19.661Z
2,2020-11-30T15:49:19.727Z,mINGiatYa7,127,2020-11-30,30,2020,11,30,15,49,19.727Z
3,2020-11-30T15:50:20.220Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,50,20.220Z
4,2020-11-30T15:51:05.403Z,mINGiatYa7,65,2020-11-30,30,2020,11,30,15,51,05.403Z
5,2020-11-30T15:51:05.405Z,yyotRBlw9E,12,2020-11-30,30,2020,11,30,15,51,05.405Z
6,2020-11-30T15:53:10.207Z,AsAxgmNPfd,87,2020-11-30,30,2020,11,30,15,53,10.207Z
7,2020-11-30T15:53:10.376Z,mINGiatYa7,188,2020-11-30,30,2020,11,30,15,53,10.376Z
8,2020-11-30T15:53:10.395Z,yyotRBlw9E,12,2020-11-30,30,2020,11,30,15,53,10.395Z
9,2020-11-30T15:54:25.289Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,15,54,25.289Z


Analizamos valores nulos

In [30]:
#Analizamos valores nulos
vars_with_na = [
    var for var in data.columns
    if data[var].isnull().sum() > 0 or data[var].dtypes != 'O'
]

# print percentage of missing values per variable
data[vars_with_na].isnull().mean()

value    0.0
Fecha    0.0
day      0.0
mes      0.0
dia      0.0
hora     0.0
dtype: float64

Armamos modelo estadístico 

In [31]:
statistics=pd.DataFrame(data.groupby(['_p_sensor','mes','dia','hora']).agg(['mean','median','min', 'max', 'std'])).reset_index(inplace=False)
statistics.columns = [' '.join(col).strip() for col in statistics.columns.values]
statistics.head(20)

,_p_sensor,mes,dia,hora,value mean,value median,value min,value max,value std,day mean,day median,day min,day max,day std
0,AsAxgmNPfd,11,30,15,86.250000,86.0,86,87,0.500000,30,30,30,30,0.0
1,AsAxgmNPfd,11,30,16,77.789474,78.0,66,86,5.912125,30,30,30,30,0.0
2,AsAxgmNPfd,11,30,17,80.428571,82.0,68,86,5.258170,30,30,30,30,0.0
3,AsAxgmNPfd,11,30,18,88.777778,89.0,87,90,0.697982,30,30,30,30,0.0
4,AsAxgmNPfd,11,30,19,88.615385,89.0,86,91,1.386750,30,30,30,30,0.0
5,AsAxgmNPfd,11,30,20,86.366667,88.0,75,91,4.350849,30,30,30,30,0.0
6,AsAxgmNPfd,11,30,21,83.555556,83.5,78,90,3.239866,30,30,30,30,0.0
7,AsAxgmNPfd,11,30,22,89.888889,90.0,89,90,0.333333,30,30,30,30,0.0
8,AsAxgmNPfd,11,30,23,85.428571,87.0,80,88,3.154739,30,30,30,30,0.0
9,AsAxgmNPfd,12,1,0,83.222222,83.0,76,91,4.734797,1,1,1,1,0.0


In [32]:
statistics.columns

Index(['_p_sensor', 'mes', 'dia', 'hora', 'value mean', 'value median',
       'value min', 'value max', 'value std', 'day mean', 'day median',
       'day min', 'day max', 'day std'],
      dtype='object')

In [33]:
statistics['hora_menos_uno']= np.where(statistics['hora']!=23,statistics['hora']+1,0)
statistics['dia_menos_uno']= np.where((statistics['hora_menos_uno']==0) & (statistics['dia']!=1) ,statistics['dia']+1,statistics['dia'])


In [24]:
statistics.head(10)

,_p_sensor,mes,dia,hora,value mean,value median,value min,value max,value std,day mean,day median,day min,day max,day std,hora_menos_uno,dia_menos_uno,mes_menos_uno
0,AsAxgmNPfd,11,30,15,86.250000,86.0,86,87,0.500000,30,30,30,30,0.0,16,30,12
1,AsAxgmNPfd,11,30,16,77.789474,78.0,66,86,5.912125,30,30,30,30,0.0,17,30,12
2,AsAxgmNPfd,11,30,17,80.428571,82.0,68,86,5.258170,30,30,30,30,0.0,18,30,12
3,AsAxgmNPfd,11,30,18,88.777778,89.0,87,90,0.697982,30,30,30,30,0.0,19,30,12
4,AsAxgmNPfd,11,30,19,88.615385,89.0,86,91,1.386750,30,30,30,30,0.0,20,30,12
5,AsAxgmNPfd,11,30,20,86.366667,88.0,75,91,4.350849,30,30,30,30,0.0,21,30,12
6,AsAxgmNPfd,11,30,21,83.555556,83.5,78,90,3.239866,30,30,30,30,0.0,22,30,12
7,AsAxgmNPfd,11,30,22,89.888889,90.0,89,90,0.333333,30,30,30,30,0.0,23,30,12
8,AsAxgmNPfd,11,30,23,85.428571,87.0,80,88,3.154739,30,30,30,30,0.0,0,31,12
9,AsAxgmNPfd,12,1,0,83.222222,83.0,76,91,4.734797,1,1,1,1,0.0,1,1,13


In [34]:
data['hora_menos_uno']=data['hora']
data['dia_menos_uno']=data['dia']

In [35]:
# Join de data estadística
data = pd.merge(data,
                 statistics,
                 on=['_p_sensor','dia_menos_uno','hora_menos_uno','mes'])


In [36]:
# Cant de sensores 
_p_sensor=pd.DataFrame(data['_p_sensor'].unique()).count()
_p_sensor[0]

3

In [ ]:
data.columns

Index(['time', '_p_sensor', 'value', 'anio', 'mes', 'dia_x', 'hora_x',
       'minutos', 'segundos', 'hora_menos_uno', 'dia_menos_uno', 'dia_y',
       'hora_y', 'value mean', 'value median', 'value min', 'value max',
       'value std'],
      dtype='object')

In [37]:
data.head(5)

,time,_p_sensor,value,Fecha,day,anio,mes,dia_x,hora_x,minutos,segundos,hora_menos_uno,dia_menos_uno,dia_y,hora_y,value mean,value median,value min,value max,value std,day mean,day median,day min,day max,day std
0,2020-11-30T16:30:43.901Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,16,30,43.901Z,16,30,30,15,86.25,86.0,86,87,0.5,30,30,30,30,0.0
1,2020-11-30T16:31:44.172Z,AsAxgmNPfd,86,2020-11-30,30,2020,11,30,16,31,44.172Z,16,30,30,15,86.25,86.0,86,87,0.5,30,30,30,30,0.0
2,2020-11-30T16:32:45.365Z,AsAxgmNPfd,85,2020-11-30,30,2020,11,30,16,32,45.365Z,16,30,30,15,86.25,86.0,86,87,0.5,30,30,30,30,0.0
3,2020-11-30T16:33:46.748Z,AsAxgmNPfd,84,2020-11-30,30,2020,11,30,16,33,46.748Z,16,30,30,15,86.25,86.0,86,87,0.5,30,30,30,30,0.0
4,2020-11-30T16:34:47.897Z,AsAxgmNPfd,84,2020-11-30,30,2020,11,30,16,34,47.897Z,16,30,30,15,86.25,86.0,86,87,0.5,30,30,30,30,0.0


Creación de filtros duros

In [38]:
# modificando los porcentajes se cambian los valores, los propuestos estan a modo de ejemplo

result = [] 
for dato,maximo,minimo,mediana,promedio in zip(data["value"],data["value max"],data["value min"],data["value median"],data['value mean']):
    if (dato*0.20) > maximo or (dato*0.20)>mediana  : 
        result.append("1") 
    elif (minimo*1.80) > dato or (dato*1.80)<mediana: 
        result.append("1") 
    else: 
        result.append("0") 
       
data["value_alerta"] = result 


Modelamos, entenamos y aplicamos el modelo no supervisado

In [39]:
data_valor=data.loc[:,['_p_sensor','time','value']]

In [ ]:
data_valor.head(5)

,_p_sensor,time,value
0,AsAxgmNPfd,2020-11-30T16:30:43.901Z,86
1,AsAxgmNPfd,2020-11-30T16:31:44.172Z,86
2,AsAxgmNPfd,2020-11-30T16:32:45.365Z,85
3,AsAxgmNPfd,2020-11-30T16:33:46.748Z,84
4,AsAxgmNPfd,2020-11-30T16:34:47.897Z,84


In [40]:
# Isolate Forest
# create a dataframe 
df = pd.DataFrame(data) 
  
# group by "X" column 
groups = df.groupby('_p_sensor') 
  
# extract keys from groups 
keys = groups.groups.keys() 

model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)


In [41]:

prediccion=[]
for nodos in keys:
  
  entrena=data_valor[data_valor['_p_sensor']==nodos]
  model.fit(entrena[['value']])
  entrena['scores_p_sensor']=model.decision_function(entrena[['value']])
  prediccion.append(entrena)

datos_inicio=pd.DataFrame(prediccion[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [ ]:
datos_inicio

In [42]:
loop=[1,2,3]
for i in range(_p_sensor[0]):
  datos_inicio=datos_inicio.append(pd.DataFrame(prediccion[i]),ignore_index = True)
  #datos_finales=pd.concat(datos_inicio.reset_index(drop=True),pd.DataFrame(prediccion[i]).reset_index(drop=True))
data_valor.shape

(7607, 3)

In [45]:
data_final = pd.merge(data,
                 datos_inicio,
                 on=['time','_p_sensor']).drop_duplicates()


In [48]:
# Ejemplo de las primeras 5 columnas.
data_final.loc[:,['time','_p_sensor','value_x','value mean','value median','value min','value max','value_alerta','scores_p_sensor']].head(5)

,time,_p_sensor,value_x,value mean,value median,value min,value max,value_alerta,scores_p_sensor
0,2020-11-30T16:30:43.901Z,AsAxgmNPfd,86,86.25,86.0,86,87,1,0.155188
2,2020-11-30T16:31:44.172Z,AsAxgmNPfd,86,86.25,86.0,86,87,1,0.155188
4,2020-11-30T16:32:45.365Z,AsAxgmNPfd,85,86.25,86.0,86,87,1,0.134951
6,2020-11-30T16:33:46.748Z,AsAxgmNPfd,84,86.25,86.0,86,87,1,0.119776
8,2020-11-30T16:34:47.897Z,AsAxgmNPfd,84,86.25,86.0,86,87,1,0.119776


In [49]:
# Al ejecutar esta línea se guarda un csv con los resultados.
data_final.loc[:,['time','_p_sensor','value_x','value mean','value median','value min','value max','value_alerta','scores_p_sensor']].to_csv('1er_prototipo_anomalias.csv',encoding='latin-1',decimal=',')